<a href="https://colab.research.google.com/github/dariomasip/elcheto-staff/blob/main/Generador_de_c%C3%B3digos_QR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalación de gestor paquetes

In [1]:
pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [2]:
pip install --upgrade qrcode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 953.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 2.6 MB/s eta 0:00:00


##Importación de paquetes necesarios

In [3]:
from PIL import Image, ImageDraw
import qrcode as qrcode
import os


In [28]:
import subprocess
import tkinter as tk
from tkinter import filedialog
from google.colab import files
from datetime import datetime

In [30]:
import random
from termcolor import colored
from datetime import datetime
import zipfile
import shutil
import csv

##Generador de letras aleatorias

In [8]:
def generate_random_letters(number_of_letters):
  """
  Esta función genera un número aleatorio de letras.

  Args:
    number_of_letters: El número de letras que se deben generar.

  Returns:
    Una cadena de letras aleatorias.
  """

  letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
  random_letters = ""
  for i in range(number_of_letters):
    random_letters += letters[random.randint(0, len(letters) - 1)]

  return random_letters

#Almacenador de códigos

Los almacena en esta variable para evitar el duplicado.



In [10]:
codes_list = {}

##Función generadora de códigos QR

In [29]:
# Función para solicitar una fecha en el formato dd-mm-yyyy
def input_fecha():
    while True:
        try:
            fecha_str = input("Ingrese la fecha (dd-mm-yyyy): ")
            fecha = datetime.strptime(fecha_str, "%d-%m-%Y")
            return fecha
        except ValueError:
            print(colored("Error: Debe ingresar una fecha válida en el formato 'dd-mm-yyyy'. Por ejemplo, '10-11-2023'.", "red"))

# Función para solicitar tipos de entradas y sus imágenes de fondo
def input_tipos_entradas():
    tipos_entradas = {}
    while True:
        tipo = input("Ingrese un tipo de entrada (o 'listo' para finalizar): ")
        if tipo.lower() == 'listo':
            if tipos_entradas:
                return tipos_entradas
            else:
                print(colored("Debe ingresar al menos un tipo de entrada.", "red"))
        else:
            # Solicita una imagen de fondo
            print(f"Por favor, seleccione la imagen de fondo para '{tipo}':")
            uploaded_background = files.upload()
            background_image_name = list(uploaded_background.keys())[0]
            tipos_entradas[tipo] = background_image_name

# Función para solicitar la cantidad de personas y sus nombres
def input_personas():
    personas = []
    while True:
        try:
            num_personas = int(input("Ingrese la cantidad de personas: "))
            for i in range(num_personas):
                nombre = input(f"Ingrese el nombre de la persona {i + 1}: ")
                personas.append({"nombre": nombre, "entradas": {}})
            return personas
        except ValueError:
            print(colored("Error: Debe ingresar un número válido.", "red"))

# Función para solicitar la cantidad de códigos para cada tipo de entrada por persona
def input_cantidades_entradas(personas, tipos_entradas):
    for persona in personas:
        print(colored(f"Ingrese la cantidad de códigos para {persona['nombre']}:", "green"))
        for tipo_entrada in tipos_entradas:
            while True:
                try:
                    cantidad = int(input(f"{tipo_entrada}: "))
                    persona['entradas'][tipo_entrada] = cantidad
                    break
                except ValueError:
                    print(colored("Error: Debe ingresar un número válido.", "red"))

# Solicita la fecha del concierto
fecha_concierto = input_fecha()
fecha_concierto_str = fecha_concierto.strftime("%d-%m-%Y")

# Solicita los tipos de entradas y sus imágenes de fondo
tipos_entradas = input_tipos_entradas()

# Solicita la cantidad de personas y sus nombres
personas = input_personas()

# Solicita la cantidad de códigos para cada tipo de entrada por persona
input_cantidades_entradas(personas, tipos_entradas)

# Genera los códigos QR y almacénalos en una carpeta temporal
temp_folder = "temp_cupones"
if not os.path.exists(temp_folder):
    os.makedirs(temp_folder)

# Genera los códigos QR para cada persona y guárdalos en un CSV
csv_file_path = f"codigos {fecha_concierto_str}.csv"

# Abre el archivo CSV en modo de escritura
with open(csv_file_path, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Escribe la fila de encabezado en el CSV
    csv_writer.writerow(["Nombre", "Tipo de Entrada", "Código QR"])

    # Genera los códigos QR para cada persona
    for persona in personas:
        nombre = persona["nombre"]
        entradas = persona["entradas"]

        for tipo_entrada, cantidad in entradas.items():
            user_directory = f"{temp_folder}/Cupones_{nombre} {fecha_concierto_str} El Cheto/{tipo_entrada}"
            user_directory_rar = f"{temp_folder}/Cupones_{nombre} {fecha_concierto_str} El Cheto"

            if not os.path.exists(user_directory):
                os.makedirs(user_directory)

            for n in range(1, cantidad + 1):
                random_number = str(random.randint(1, 99))
                random_letters = generate_random_letters(4)
                file_name = f"{random_letters}{random_number}"

                while file_name in codes_list:
                    random_number = str(random.randint(1, 99))
                    random_letters = generate_random_letters(4)
                    file_name = f"{random_letters}{random_number}"

                codes_list[file_name] = 1

                qr_code = qrcode.QRCode(version=2, box_size=6, border=0)
                qr_code.add_data(file_name)

                img = qr_code.make_image(fill_color='#d5d4d5', back_color='#101010').resize((170, 170))

                image = Image.open(tipos_entradas[tipo_entrada])  # Usa la imagen de fondo correspondiente al tipo de entrada

                # Calcula la coordenada X para centrar horizontalmente el código QR
                background_width = image.size[0]
                qr_x = int((background_width - 170) / 2)

                # Coloca el código QR en las coordenadas seleccionadas por el usuario
                image.paste(img, (qr_x, 35))

                draw = ImageDraw.Draw(image)
                draw.text((185, 215), file_name, fill="#d5d4d5", align="center", font_size=30, font_family="Arial")

                # Guarda el archivo y descárgalo automáticamente
                file_path = f"{user_directory}/#{n}__[{tipo_entrada}] cupon_{file_name}.png"
                image.save(file_path)

                # Escribe la información en el CSV
                csv_writer.writerow([nombre, tipo_entrada, file_name])

        # Crea un archivo ZIP por persona
        zip_file_path = f"Cupones_{nombre} {fecha_concierto_str} El Cheto.zip"
        with zipfile.ZipFile(zip_file_path, "w") as zipf:
            for root, dirs, files in os.walk(user_directory_rar):
                for file in files:
                    zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), user_directory_rar))

        # Limpia la carpeta temporal de esa persona
        shutil.rmtree(user_directory_rar)

Ingrese la fecha (dd-mm-yyyy): 10-11-2023
Ingrese un tipo de entrada (o 'listo' para finalizar): Palco VIP
Por favor, seleccione la imagen de fondo para 'Palco VIP':


Saving [2]Palco VIP Hombre - El Cheto.png to [2]Palco VIP Hombre - El Cheto (3).png
Ingrese un tipo de entrada (o 'listo' para finalizar): Mujer
Por favor, seleccione la imagen de fondo para 'Mujer':


Saving Cupon MUJER 2 - El Cheto.png to Cupon MUJER 2 - El Cheto (1).png
Ingrese un tipo de entrada (o 'listo' para finalizar): listo
Ingrese la cantidad de personas: 2
Ingrese el nombre de la persona 1: Dario Masipi
Ingrese el nombre de la persona 2: Maria Chavez
Ingrese la cantidad de códigos para Dario Masipi:
Palco VIP: 10
Mujer: 15
Ingrese la cantidad de códigos para Maria Chavez:
Palco VIP: 10
Mujer: 10
